In [7]:
# The chatbot will be trained on the dataset which contains categories, pattern and responses. 
# We use a special recirrent neural networl (LTSM) to classify which category the user's message belongs to and then we will give a random response from the list of responses.

In [8]:
!pip install tensorflow
!pip install keras
!pip install pickle
!pip install nltk

ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle


In [9]:
# The 5 steps to create a chatbot
# 1. Import and load the data file
# 2. Preprocess data
# 3. Create training and testing data
# 4. Build the model
# 5. Predict the response

In [10]:
# Import and load the data file

import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [11]:
lemmatizer = WordNetLemmatizer()
words = []
classes = []
documents = []
ignore_words = ['?', '!', ',', '.']
data_file = open('my_intents.json').read()
intents = json.loads(data_file)

In [12]:
# 2. Preprocess data

for intent in intents['intents']:
  for pattern in intent['patterns']:

    # tokenize each word
    w = nltk.word_tokenize(pattern)
    words.extend(w)

    # add documents in the corpus
    documents.append((w, intent['tag']))

    # add to classes list
    if intent['tag'] not in classes:
      classes.append(intent['tag'])

In [13]:
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words] # lemmatization of words
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

In [14]:
# 3. Create training and testing data

# create training data
training = []
# create an empty array for output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
  # initialize bag of words
  bag = []
  # list of tokenized words for the pattern
  pattern_words = doc[0]
  # lemmatize each word
  pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
  # create bag of words array with 1, if word match found in current patter
  for w in words:
    bag.append(1) if w in pattern_words else bag.append(0)
  # 1 for current tag
  output_row = list(output_empty)
  output_row[classes.index(doc[1])] = 1

  training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)
# train_x - patterns, train_y = intents
train_x = list(training[:,0])
train_y = list(training[:,1])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [15]:
from tensorflow.keras import optimizers
# Build the model
# There are 3 layers. First - 128 neurons, second - 64 neurons
# 3rd output layer contains number of neurons equal to number of intents
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]), ), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd,
              metrics=['accuracy'])

# fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=210,
                 batch_size=5, verbose=1)

model.save('chat_bot.h5', hist)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/210
9/9 [==============================] - 1s 2ms/step - loss: 2.1650 - accuracy: 0.1860
Epoch 2/210
9/9 [==============================] - 0s 2ms/step - loss: 2.1649 - accuracy: 0.1860
Epoch 3/210
9/9 [==============================] - 0s 2ms/step - loss: 2.0564 - accuracy: 0.2326
Epoch 4/210
9/9 [==============================] - 0s 2ms/step - loss: 2.0182 - accuracy: 0.3256
Epoch 5/210
9/9 [==============================] - 0s 2ms/step - loss: 1.8923 - accuracy: 0.3721
Epoch 6/210
9/9 [==============================] - 0s 2ms/step - loss: 1.9556 - accuracy: 0.3953
Epoch 7/210
9/9 [==============================] - 0s 2ms/step - loss: 1.8354 - accuracy: 0.3953
Epoch 8/210
9/9 [==============================] - 0s 2ms/step - loss: 1.6803 - accuracy: 0.5116
Epoch 9/210
9/9 [==============================] - 0s 2ms/step - loss: 1.6907 - accuracy: 0.4651
Epoch 10/210
9/9 [==============================] - 0s 2ms/step - loss: 1.4844 - accuracy: 0.5814
Epoch 11/210
9/9 [===========

In [16]:
# 5. Testing. Predict the response

from keras.models import load_model
from nltk import probability

model = load_model('chat_bot.h5')
intents = json.loads(open('my_intents.json').read())
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))

In [17]:
def clean_up_sentence(sentence):
  # split words into array
  sentence_words = nltk.word_tokenize(sentence)
  # create short form for word
  sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
  # return bag of words array
  return sentence_words

def bow(sentence, words, show_details=True):
  # tokenize the pattern
  sentence_words = clean_up_sentence(sentence)
  # bag of words
  bag = [0] * len(words)
  for s in sentence_words:
    for i,w in enumerate(words):
      if w == s:
        bag[i] = 1
  return (np.array(bag))

def predict_class(sentence, model):
  # filter out predictions below a threshold
  p = bow(sentence, words)
  res = model.predict(np.array([p]))[0]
  ERROR_THRESHOLD = 0.25
  results = [[i, r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
  # sort by strength of probability
  results.sort(key=lambda x: x[1], reverse=True)
  return_list = []

  for r in results:
    return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})

  return return_list

In [18]:
def getResponse(ints, intents_json):
  tag = ints[0]['intent']
  list_of_intents = intents_json['intents']
  result = ''
  for i in list_of_intents:
    if (i['tag'] == tag):
      result = random.choice(i['responses'])
      break
  return result

# get response from our neurol network
def chatbot_response(text):
  ints = predict_class(text, model)
  res = getResponse(ints, intents)
  return res

In [19]:
chatbot_response('Do you have creator?')

'My creator is Artem Meller!'